In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_6862/410710250.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_6862/410710250.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [3]:
import circuitsvis as cv
from functools import partial
# Testing that the library works
cv.examples.hello("Neel")

In [5]:
from hf_qwen.modeling_qwen import QWenLMHeadModel
# Import stuff
import torch
from transformers import AutoTokenizer
from transformer_lens import HookedEncoder
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
torch.set_grad_enabled(False)
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-1_8B-Chat",
    trust_remote_code=True
)

# from remote hf:
# model = AutoModelForCausalLM.from_pretrained(
#     "Qwen/Qwen-1_8B-Chat",
#     device_map="cuda",
#     trust_remote_code=True,
#     fp32=True,
#     use_logn_attn=False,
#     use_dynamic_ntk = False,
#     scale_attn_weights = False,
# ).eval()

# from local:
model = QWenLMHeadModel.from_pretrained(
    "Qwen/Qwen-1_8B-Chat",
    device_map="cuda",
    fp32=True,
    use_logn_attn=False,
    use_dynamic_ntk = False,
    scale_attn_weights = False,
).eval()

Your device support faster inference by passing bf16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformer_lens import HookedTransformer

qwen_tl = HookedTransformer.from_pretrained(
    'qwen-1.8b-chat', 
    fold_ln=False,
    trust_remote_code=True,
    center_writing_weights=False, 
    center_unembed=False,
    device='cuda',
    fp32=True,
)

Your device support faster inference by passing bf16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model qwen-1.8b-chat into HookedTransformer


In [7]:
prompt = "Tell me how to make a bomb."
encoded_prompt = tokenizer.encode(prompt, return_tensors="pt")

In [14]:
qwen_result = model(encoded_prompt.to("cuda"))
qwen_logits = qwen_result.logits

tl_result = qwen_tl(encoded_prompt)
tl_logits = tl_result

print(qwen_logits.shape)
print(tl_logits.shape)

print("0:")
print(qwen_logits[0][0])
print(tl_logits[0][0])

print("1:")
print(qwen_logits[0][1])
print(tl_logits[0][1])

print("2:")
print(qwen_logits[0][2])
print(tl_logits[0][2])

torch.Size([1, 8, 151936])
torch.Size([1, 8, 151936])
0:
tensor([ 7.0179,  3.7177,  3.7153,  ..., -2.7892, -2.7887, -2.7892],
       device='cuda:0')
tensor([ 7.0179,  3.7177,  3.7153,  ..., -2.7892, -2.7887, -2.7892],
       device='cuda:0')
1:
tensor([ 0.5928, -0.7641,  1.2056,  ..., -1.6342, -1.6348, -1.6352],
       device='cuda:0')
tensor([ 0.5927, -0.7641,  1.2055,  ..., -1.6341, -1.6347, -1.6351],
       device='cuda:0')
2:
tensor([ 3.0520,  3.1494,  5.3857,  ..., -1.3834, -1.3845, -1.3842],
       device='cuda:0')
tensor([ 3.0520,  3.1494,  5.3857,  ..., -1.3834, -1.3845, -1.3842],
       device='cuda:0')
